# Data Governance Load Engine - Execução Manual

Este notebook demonstra como executar manualmente o motor de load de dados para governança, incluindo configuração, teste de conectividade e sincronização de dados.

## Pré-requisitos

1. Ambiente Databricks configurado com Unity Catalog
2. Service Principal configurado no Azure
3. Banco de dados PostgreSQL disponível
4. Variáveis de ambiente configuradas

## 1. Configuração Inicial

In [ ]:
# Importações necessárias
import os
import sys
import asyncio
import json
from datetime import datetime
from pprint import pprint

# Adiciona o diretório src ao path
sys.path.append('../src')

# Carrega variáveis de ambiente
from dotenv import load_dotenv
load_dotenv('../.env')

print("✅ Configuração inicial concluída")

## 2. Validação da Configuração

In [ ]:
# Valida configurações
from config import validate_configuration, get_settings

if validate_configuration():
    print("✅ Configuração válida")
    settings = get_settings()
    
    print(f"\n📊 Configurações carregadas:")
    print(f"  - Databricks Host: {settings.databricks.host}")
    print(f"  - Unity Catalog: {settings.databricks.catalog_name}")
    print(f"  - Azure Tenant: {settings.azure.tenant_id}")
    print(f"  - Database URL: {settings.database.url.split('@')[1] if '@' in settings.database.url else 'configurado'}")
    
    print(f"\n🎛️ Feature Flags:")
    print(f"  - Unity Catalog Sync: {settings.features.enable_unity_catalog_sync}")
    print(f"  - Azure Sync: {settings.features.enable_azure_sync}")
    print(f"  - Quality Inference: {settings.features.enable_quality_inference}")
    print(f"  - Anomaly Detection: {settings.features.enable_anomaly_detection}")
    print(f"  - Auto Classification: {settings.features.enable_auto_classification}")
else:
    print("❌ Configuração inválida - verifique o arquivo .env")

## 3. Teste de Conectividade

In [ ]:
# Testa conectividade com Unity Catalog
from connectors import UnityCatalogConnector

async def test_unity_catalog():
    settings = get_settings()
    
    config = {
        "host": settings.databricks.host,
        "token": settings.databricks.token,
        "catalog_name": settings.databricks.catalog_name,
        "schema_name": settings.databricks.schema_name,
        "warehouse_id": settings.databricks.warehouse_id
    }
    
    connector = UnityCatalogConnector(config)
    
    try:
        print("🔌 Testando conexão Unity Catalog...")
        connected = await connector.connect()
        
        if connected:
            print("✅ Conexão Unity Catalog estabelecida")
            
            # Obtém metadados básicos
            metadata = await connector.get_metadata()
            catalogs = metadata.get('catalogs', [])
            
            print(f"📊 Encontrados {len(catalogs)} catálogos:")
            for catalog in catalogs[:5]:  # Mostra apenas os primeiros 5
                print(f"  - {catalog.get('name', 'N/A')}: {catalog.get('comment', 'Sem descrição')}")
            
            await connector.disconnect()
            return True
        else:
            print("❌ Falha na conexão Unity Catalog")
            return False
            
    except Exception as e:
        print(f"❌ Erro na conexão Unity Catalog: {e}")
        return False

# Executa teste
uc_result = await test_unity_catalog()

In [ ]:
# Testa conectividade com Azure
from connectors import AzureConnector

async def test_azure():
    settings = get_settings()
    
    config = {
        "tenant_id": settings.azure.tenant_id,
        "client_id": settings.azure.client_id,
        "client_secret": settings.azure.client_secret,
        "subscription_id": settings.azure.subscription_id,
        "resource_group": settings.azure.resource_group,
        "data_factory_name": settings.azure.data_factory_name,
        "synapse_workspace": settings.azure.synapse_workspace
    }
    
    connector = AzureConnector(config)
    
    try:
        print("🔌 Testando conexão Azure...")
        connected = await connector.connect()
        
        if connected:
            print("✅ Conexão Azure estabelecida")
            
            # Obtém metadados básicos
            metadata = await connector.get_metadata()
            resource_groups = metadata.get('resource_groups', [])
            
            print(f"📊 Encontrados {len(resource_groups)} resource groups:")
            for rg in resource_groups[:5]:  # Mostra apenas os primeiros 5
                print(f"  - {rg.get('name', 'N/A')} ({rg.get('location', 'N/A')})")
            
            await connector.disconnect()
            return True
        else:
            print("❌ Falha na conexão Azure")
            return False
            
    except Exception as e:
        print(f"❌ Erro na conexão Azure: {e}")
        return False

# Executa teste
azure_result = await test_azure()

## 4. Inicialização do Motor de Load

In [ ]:
# Inicializa o motor de load
from load_engine import DataGovernanceLoadEngine

async def initialize_engine():
    print("🚀 Inicializando Data Governance Load Engine...")
    
    engine = DataGovernanceLoadEngine()
    
    try:
        initialized = await engine.initialize()
        
        if initialized:
            print("✅ Motor de load inicializado com sucesso")
            
            # Obtém status
            status = await engine.get_status()
            
            print(f"\n📊 Status do Motor:")
            print(f"  - Status: {status['engine_status']}")
            print(f"  - Conectores: {list(status['connectors'].keys())}")
            
            print(f"\n🎛️ Configurações Ativas:")
            settings_info = status['settings']
            for key, value in settings_info.items():
                print(f"  - {key}: {value}")
            
            return engine
        else:
            print("❌ Falha na inicialização do motor")
            return None
            
    except Exception as e:
        print(f"❌ Erro na inicialização: {e}")
        return None

# Inicializa motor
engine = await initialize_engine()

## 5. Sincronização de Dados

### 5.1 Sincronização Unity Catalog

In [ ]:
# Sincroniza dados do Unity Catalog
if engine and uc_result:
    print("🔄 Iniciando sincronização Unity Catalog...")
    start_time = datetime.now()
    
    try:
        result = await engine.sync_connector("unity_catalog")
        
        if result.get('success', False):
            print("✅ Sincronização Unity Catalog concluída")
            
            print(f"\n📊 Resultados:")
            print(f"  - Registros brutos: {result.get('raw_records', 0)}")
            print(f"  - Registros transformados: {result.get('transformed_records', 0)}")
            print(f"  - Registros carregados: {result.get('loaded_records', 0)}")
            print(f"  - Duração: {result.get('duration_seconds', 0):.2f}s")
            
            if result.get('errors'):
                print(f"\n⚠️ Erros encontrados:")
                for error in result['errors'][:3]:  # Mostra apenas os primeiros 3
                    print(f"  - {error}")
        else:
            print("❌ Falha na sincronização Unity Catalog")
            if result.get('error'):
                print(f"Erro: {result['error']}")
                
    except Exception as e:
        print(f"❌ Erro na sincronização Unity Catalog: {e}")
else:
    print("⏭️ Pulando sincronização Unity Catalog (motor não inicializado ou conexão falhou)")

### 5.2 Sincronização Azure

In [ ]:
# Sincroniza dados do Azure
if engine and azure_result:
    print("🔄 Iniciando sincronização Azure...")
    start_time = datetime.now()
    
    try:
        result = await engine.sync_connector("azure")
        
        if result.get('success', False):
            print("✅ Sincronização Azure concluída")
            
            print(f"\n📊 Resultados:")
            print(f"  - Registros brutos: {result.get('raw_records', 0)}")
            print(f"  - Registros transformados: {result.get('transformed_records', 0)}")
            print(f"  - Registros carregados: {result.get('loaded_records', 0)}")
            print(f"  - Duração: {result.get('duration_seconds', 0):.2f}s")
            
            if result.get('errors'):
                print(f"\n⚠️ Erros encontrados:")
                for error in result['errors'][:3]:  # Mostra apenas os primeiros 3
                    print(f"  - {error}")
        else:
            print("❌ Falha na sincronização Azure")
            if result.get('error'):
                print(f"Erro: {result['error']}")
                
    except Exception as e:
        print(f"❌ Erro na sincronização Azure: {e}")
else:
    print("⏭️ Pulando sincronização Azure (motor não inicializado ou conexão falhou)")

### 5.3 Sincronização Completa

In [ ]:
# Executa sincronização completa
if engine:
    print("🔄 Iniciando sincronização completa...")
    start_time = datetime.now()
    
    try:
        result = await engine.sync_all()
        
        if result.get('success', False):
            print("✅ Sincronização completa concluída")
            
            print(f"\n📊 Resumo Geral:")
            print(f"  - Sync ID: {result.get('sync_id')}")
            print(f"  - Total de registros: {result.get('total_records', 0)}")
            print(f"  - Duração total: {result.get('duration_seconds', 0):.2f}s")
            
            print(f"\n📊 Por Conector:")
            for connector, data in result.get('connectors', {}).items():
                if 'error' not in data:
                    print(f"  - {connector}:")
                    print(f"    • Registros brutos: {data.get('raw_records', 0)}")
                    print(f"    • Registros carregados: {data.get('loaded_records', 0)}")
                    print(f"    • Duração: {data.get('duration_seconds', 0):.2f}s")
                else:
                    print(f"  - {connector}: ❌ {data['error']}")
            
            if result.get('errors'):
                print(f"\n⚠️ Erros gerais:")
                for error in result['errors']:
                    print(f"  - {error}")
        else:
            print("❌ Falha na sincronização completa")
            if result.get('errors'):
                print(f"Erros: {result['errors']}")
                
    except Exception as e:
        print(f"❌ Erro na sincronização completa: {e}")
else:
    print("⏭️ Pulando sincronização completa (motor não inicializado)")

## 6. Monitoramento e Métricas

In [ ]:
# Obtém métricas detalhadas
if engine:
    print("📊 Obtendo métricas do motor...")
    
    try:
        status = await engine.get_status()
        
        print(f"\n🎯 Métricas Operacionais:")
        metrics = status.get('metrics', {})
        print(f"  - Total de sincronizações: {metrics.get('total_syncs', 0)}")
        print(f"  - Sincronizações bem-sucedidas: {metrics.get('successful_syncs', 0)}")
        print(f"  - Sincronizações falhadas: {metrics.get('failed_syncs', 0)}")
        print(f"  - Total de registros processados: {metrics.get('total_records_processed', 0)}")
        print(f"  - Duração da última sincronização: {metrics.get('last_sync_duration', 0):.2f}s")
        
        if metrics.get('errors'):
            print(f"\n⚠️ Últimos erros ({len(metrics['errors'])})")
            for error in metrics['errors'][-3:]:  # Últimos 3 erros
                print(f"  - {error}")
        
        print(f"\n🔌 Status dos Conectores:")
        for connector_name, connector_status in status.get('connectors', {}).items():
            print(f"  - {connector_name}:")
            print(f"    • Conectado: {connector_status.get('connected', False)}")
            print(f"    • Taxa de sucesso: {connector_status.get('success_rate_percentage', 0):.1f}%")
            print(f"    • Total de requests: {connector_status.get('total_requests', 0)}")
            print(f"    • Tempo médio de resposta: {connector_status.get('avg_response_time_seconds', 0):.3f}s")
            
            if connector_status.get('last_error'):
                print(f"    • Último erro: {connector_status['last_error']}")
        
        print(f"\n🕐 Última Sincronização:")
        last_sync = status.get('last_sync', {})
        for connector, timestamp in last_sync.items():
            print(f"  - {connector}: {timestamp}")
            
    except Exception as e:
        print(f"❌ Erro ao obter métricas: {e}")
else:
    print("⏭️ Pulando métricas (motor não inicializado)")

## 7. Análise de Dados Coletados

In [ ]:
# Análise dos dados coletados (simulação)
print("📈 Análise dos dados coletados...")

# Simulação de análise baseada nas métricas
if engine:
    status = await engine.get_status()
    metrics = status.get('metrics', {})
    
    total_records = metrics.get('total_records_processed', 0)
    
    print(f"\n📊 Resumo da Coleta:")
    print(f"  - Total de registros coletados: {total_records:,}")
    
    # Estimativa de cobertura baseada no modelo de 36 tabelas
    if total_records > 0:
        # Estimativas baseadas na análise de fontes
        unity_catalog_coverage = min(85, (total_records / 10000) * 85)  # 85% máximo
        azure_coverage = min(10, (total_records / 5000) * 10)  # 10% máximo
        total_coverage = unity_catalog_coverage + azure_coverage
        
        print(f"\n📊 Estimativa de Cobertura do Modelo:")
        print(f"  - Unity Catalog: {unity_catalog_coverage:.1f}%")
        print(f"  - Azure: {azure_coverage:.1f}%")
        print(f"  - Total automático: {total_coverage:.1f}%")
        print(f"  - Restante (manual/inferível): {100 - total_coverage:.1f}%")
        
        # Tabelas estimadas por categoria
        tables_covered = int((total_coverage / 100) * 36)
        tables_remaining = 36 - tables_covered
        
        print(f"\n📋 Tabelas do Modelo (36 total):")
        print(f"  - Tabelas com dados: {tables_covered}")
        print(f"  - Tabelas pendentes: {tables_remaining}")
        
        # Recomendações
        print(f"\n💡 Recomendações:")
        if total_coverage < 50:
            print(f"  - Verificar conectividade e permissões")
            print(f"  - Configurar mais fontes de dados")
        elif total_coverage < 80:
            print(f"  - Implementar configurações manuais para tabelas restantes")
            print(f"  - Ativar inferência de qualidade e classificação automática")
        else:
            print(f"  - Excelente cobertura! Focar em qualidade dos dados")
            print(f"  - Implementar monitoramento contínuo")
    else:
        print(f"\n⚠️ Nenhum registro coletado - verificar configurações")
else:
    print("⏭️ Pulando análise (motor não inicializado)")

## 8. Limpeza e Finalização

In [ ]:
# Finaliza o motor de load
if engine:
    print("🔄 Finalizando motor de load...")
    
    try:
        await engine.shutdown()
        print("✅ Motor de load finalizado com sucesso")
        
    except Exception as e:
        print(f"❌ Erro ao finalizar motor: {e}")
else:
    print("⏭️ Motor não estava inicializado")

print("\n🎉 Execução manual concluída!")
print("\n📋 Próximos passos:")
print("  1. Revisar logs e métricas")
print("  2. Configurar sincronização automática")
print("  3. Implementar dashboards de monitoramento")
print("  4. Configurar alertas para falhas")

## 9. Troubleshooting

### Problemas Comuns

#### Erro de Autenticação Databricks
```python
# Verificar token
import requests
headers = {"Authorization": f"Bearer {os.getenv('DATABRICKS_TOKEN')}"}
response = requests.get(f"{os.getenv('DATABRICKS_HOST')}/api/2.0/clusters/list", headers=headers)
print(f"Status: {response.status_code}")
```

#### Erro de Permissões Azure
```python
# Verificar Service Principal
from azure.identity import ClientSecretCredential
credential = ClientSecretCredential(
    tenant_id=os.getenv('AZURE_TENANT_ID'),
    client_id=os.getenv('AZURE_CLIENT_ID'),
    client_secret=os.getenv('AZURE_CLIENT_SECRET')
)
token = credential.get_token("https://management.azure.com/.default")
print(f"Token obtido: {bool(token)}")
```

#### Erro de Conectividade Banco
```python
# Testar conexão PostgreSQL
import asyncpg
async def test_db():
    try:
        conn = await asyncpg.connect(os.getenv('DATABASE_URL'))
        await conn.close()
        print("✅ Conexão banco OK")
    except Exception as e:
        print(f"❌ Erro banco: {e}")

await test_db()
```